In [ ]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objects as go

import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

import time

In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Path to the .env file inside 01 - Documentation
env_path = Path("01 - Documentation") / "keys.env"

# Load the .env file from that path
load_dotenv(dotenv_path=env_path)

# Now you can access your keys
API_KEY = os.getenv("ALPACA_API_KEY")
SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

print(API_KEY[:4] + "****")  # optional: verify it's working

In [ ]:
import plotly.express as px

myAssets = ["AMZN", "META", "GOOGL", "AMD", "NKE", "UBER", "COST", "JPM", "CRM", "TXRH"]

